# Firewall Log Analysis

## Step 1
- Importing json and pandas libraries for data manipulation and analysis.
- Having a look at the entire dataset to understand its structure and contents.
- Then I use df.info() to get a concise summary of the DataFrame.

In [30]:
import json
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# Try line-delimited JSON first, then fall back to a single JSON object/array.
try:
    df = pd.read_json("lab_2_firewall_log.json", lines=True)
except ValueError:
    with open("lab_2_firewall_log.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    df = pd.DataFrame(data)

df

,@timestamp,cisco.asa.destination_interface,cisco.asa.message_id,cisco.asa.rule_name,cisco.asa.source_interface,destination.address,destination.ip,destination.port,event.action,event.category,event.code,event.dataset,event.kind,event.module,event.original,event.outcome,event.severity,event.timezone,event.type,fileset.name,input.type,log.file.path,log.level,log.offset,network.community_id,network.iana_number,network.transport,observer.egress.interface.name,observer.ingress.interface.name,observer.product,observer.type,observer.vendor,related.ip,service.type,source.address,source.ip,source.port,tags,cisco.asa.suffix,host.hostname,observer.hostname,related.hosts,cisco.asa.connection_id,cisco.asa.mapped_destination_ip,cisco.asa.mapped_destination_port,cisco.asa.mapped_source_ip,cisco.asa.mapped_source_port,destination.nat.port,network.direction,source.nat.ip,destination.nat.ip,event.duration,event.end,event.start,network.bytes,cisco.asa.destination_username,cisco.asa.source_username,destination.user.name,related.user,source.user.name,user.name,network.protocol,process.name,cisco.asa.mapped_destination_host,destination.domain,event.reason,cisco.asa.icmp_code,cisco.asa.icmp_type,server.domain,source.nat.port,cisco.asa.threat_category,cisco.asa.threat_level,url.original,url.path,url.domain,url.scheme,url.extension,cisco.asa.termination_user,destination.geo.city_name,destination.geo.continent_name,destination.geo.country_iso_code,destination.geo.country_name,destination.geo.location.lat,destination.geo.location.lon,destination.geo.region_iso_code,destination.geo.region_name,destination.as.number,destination.as.organization.name,source.as.number,source.as.organization.name,source.user.domain,source.geo.city_name,source.geo.continent_name,source.geo.country_iso_code,source.geo.country_name,source.geo.location.lat,source.geo.location.lon,source.geo.region_iso_code,source.geo.region_name,destination.user.domain,cisco.asa.termination_initiator,cisco.asa.tunnel_type,network.type,source.domain,log.syslog.facility.code,log.syslog.priority,log.syslog.severity.code,timestamp,src_ip,dest_ip,src_port,dest_port,protocol,action,bytes
0,2013-04-15T09:36:50.000-02:00,outside,106023,acl_dmz,dmz,192.0.0.8,192.0.0.8,53.0,firewall-rule,[network],106023.0,cisco.asa,event,cisco,"%ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group ""acl_dmz"" [0xe3aab522, 0x0]",success,4.0,-02:00,"[connection, denied]",asa,log,sample.log,warning,0.0,1:3NxcSu9jwJUYIYwJ2TO4TSNnPX8=,6.0,tcp,outside,dmz,asa,firewall,Cisco,"[10.1.2.30, 192.0.0.8]",cisco,10.1.2.30,10.1.2.30,63016.0,"[cisco-asa, forwarded]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-04-15T09:36:50.000-02:00,outside,106023,acl_dmz,dmz,192.0.0.8,192.0.0.8,53.0,firewall-rule,[network],106023.0,cisco.asa,event,cisco,"%ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 type 3, code 0, by access-group ""acl_dmz"" [0xe3aab522, 0x0]",success,4.0,-02:00,"[connection, denied]",asa,log,sample.log,warning,139.0,1:3NxcSu9jwJUYIYwJ2TO4TSNnPX8=,6.0,tcp,outside,dmz,asa,firewall,Cisco,"[10.1.2.30, 192.0.0.8]",cisco,10.1.2.30,10.1.2.30,63016.0,"[cisco-asa, forwarded]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-04-15T13:34:34.000Z,outside,106100,acl_in,inside,192.0.0.89,,2000.0,firewall-rule,[network],106100.0,cisco.asa,event,cisco,"%ASA-session-5-106100: access-list acl_in permitted tcp inside/10.1.2.16(2241) -> outside/192.0.0.89(2000) hit-

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Columns: 115 entries, @timestamp to bytes
dtypes: float64(24), object(91)
memory usage: 125.0+ KB


In [32]:
# Count how many times each column appears with a non-null value
column_counts = df.notna().sum().sort_values(ascending=False)
column_counts

@timestamp                           89
cisco.asa.message_id                 89
event.code                           89
event.severity                       89
event.action                         89
event.module                         89
event.dataset                        89
event.original                       89
tags                                 89
observer.vendor                      89
source.address                       89
observer.type                        89
observer.product                     89
service.type                         89
log.offset                           89
log.level                            89
log.file.path                        89
input.type                           89
fileset.name                         89
source.ip                            88
related.ip                           88
destination.address                  88
event.kind                           87
event.category                       87
event.type                           87


The above tells us the events are primarily in 2 formats
- The first 89 rows follow the first format with fields like @timestamp etc.
- The last 50 rows follow a different format with fields like timestamp

So we might need to treat them differently

In [34]:

# Normalize timestamp: use '@timestamp' if present, else 'timestamp', else NaN
import numpy as np
def get_normalized_timestamp(row):
    if pd.notna(row.get('@timestamp')) and row.get('@timestamp') != '':
        return row.get('@timestamp')
    elif pd.notna(row.get('timestamp')) and row.get('timestamp') != '':
        return row.get('timestamp')
    else:
        return np.nan

df['normalized_timestamp'] = df.apply(get_normalized_timestamp, axis=1)
df['normalized_timestamp'] = pd.to_datetime(df['normalized_timestamp'], errors='coerce')
df[['@timestamp', 'timestamp', 'normalized_timestamp', 'event.timezone']]

/tmp/ipykernel_122437/1490592367.py:12: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['normalized_timestamp'] = pd.to_datetime(df['normalized_timestamp'], errors='coerce')


,@timestamp,timestamp,normalized_timestamp,event.timezone
0,2013-04-15T09:36:50.000-02:00,NaN,2013-04-15 09:36:50-02:00,-02:00
1,2013-04-15T09:36:50.000-02:00,NaN,2013-04-15 09:36:50-02:00,-02:00
2,2014-04-15T13:34:34.000Z,NaN,2014-04-15 13:34:34+00:00,NaN
3,2013-04-24T16:00:28.000-02:00,NaN,2013-04-24 16:00:28-02:00,-02:00
4,2013-04-24T16:00:27.000-02:00,NaN,2013-04-24 16:00:27-02:00,-02:00
5,2013-04-29T12:59:50.000-02:00,NaN,2013-04-29 12:59:50-02:00,-02:00
6,2013-04-29T12:59:50.000-02:00,NaN,2013-04-29 12:59:50-02:00,-02:00
7,2013-04-29T12:59:50.000-02:00,NaN,2013-04-29 12:59:50-02:00,-02:00
8,2013-04-29T12:59:50.000-02:00,NaN,2013-04-29 12:59:50-02:00,-02:00
9,2013-04-29T12:59:50.000-02:00,NaN,2013-04-29 12:59:50-02:00,-02:00


In [ ]:
# Standardize key fields, keeping all unique values as arrays if they differ
def combine_fields(row, fields):
    values = []
    for f in fields:
        v = row.get(f)
        if v is None or (isinstance(v, float) and pd.isna(v)):
            continue
        if isinstance(v, list):
            values.extend([item for item in v if item not in [None, '', float('nan')]])
        elif v != '' and v is not None:
            values.append(v)
    # Remove duplicates, preserve order
    values = list(dict.fromkeys(values))
    return values if len(values) > 1 else (values[0] if values else None)

df['src_ip'] = df.apply(lambda r: combine_fields(r, ['source.ip', 'source.address']), axis=1)
df['dest_ip'] = df.apply(lambda r: combine_fields(r, ['destination.ip', 'destination.address']), axis=1)
df['src_port'] = df.apply(lambda r: combine_fields(r, ['source.port', 'src_port']), axis=1)
df['dest_port'] = df.apply(lambda r: combine_fields(r, ['destination.port', 'dest_port']), axis=1)
# df['action'] = df.apply(lambda r: combine_fields(r, ['event.action', 'event.type', 'event.outcome', 'action']), axis=1)
# Might not be a good idea to combine this, seems like they are diff actions
df['action'] = df.apply(lambda r: combine_fields(r, ['event.action', 'action']), axis=1) 
df['bytes'] = df.apply(lambda r: combine_fields(r, ['bytes', 'event.bytes', 'network.bytes']), axis=1)
df['protocol'] = df.apply(lambda r: combine_fields(r, ['network.protocol', 'protocol', 'network.transport']), axis=1)
df['url'] = df.apply(lambda r: combine_fields(r, ['url.original', 'url.path', 'url.scheme']), axis=1)
df['user'] = df.apply(lambda r: combine_fields(r, ['user.name', 'user.id']), axis=1)
df[['src_ip', 'dest_ip', 'src_port', 'dest_port', 'bytes', 'protocol', 'url', 'user', 'event.type', 'action', 'event.outcome']]

In [ ]:
# Analyze most common IPs: total, as source, as destination, and percent of all rows
from collections import Counter

def flatten_ips(col):
    # Flatten lists and single values into a list of IPs
    ips = []
    for v in col:
        if isinstance(v, list):
            ips.extend(v)
        elif pd.notna(v):
            ips.append(v)
    return ips

src_ips = flatten_ips(df['src_ip'])
dest_ips = flatten_ips(df['dest_ip'])
all_ips = src_ips + dest_ips
total_rows = len(df)
ip_counter = Counter(all_ips)
src_counter = Counter(src_ips)
dest_counter = Counter(dest_ips)
ip_stats = []

for ip, count in ip_counter.most_common():
    src_count = src_counter.get(ip, 0)
    dest_count = dest_counter.get(ip, 0)
    percent = (count / total_rows) * 100
    ip_stats.append({'ip': ip, 'total': count, 'source': src_count, 'destination': dest_count, 'percent': percent})
ip_stats_df = pd.DataFrame(ip_stats)
ip_stats_df.sort_values('total', ascending=False)

In [ ]:
# Analyze most common ports: total, as source, as destination, and percent of all rows
def flatten_ports(col):
    ports = []
    for v in col:
        if isinstance(v, list):
            ports.extend(v)
        elif pd.notna(v):
            ports.append(v)
    return ports

src_ports = flatten_ports(df['src_port'])
dest_ports = flatten_ports(df['dest_port'])
all_ports = src_ports + dest_ports
port_counter = Counter(all_ports)
src_port_counter = Counter(src_ports)
dest_port_counter = Counter(dest_ports)
port_stats = []

for port, count in port_counter.most_common():
    src_count = src_port_counter.get(port, 0)
    dest_count = dest_port_counter.get(port, 0)
    percent = (count / total_rows) * 100
    port_stats.append({'port': port, 'total': count, 'source': src_count, 'destination': dest_count, 'percent': percent})
port_stats_df = pd.DataFrame(port_stats)
port_stats_df.sort_values('total', ascending=False).head(20)

In [ ]:
# All rows for IP 192.0.2.222 (as source or destination)
mask_222 = df['src_ip'].apply(lambda x: '192.0.2.222' in x if isinstance(x, list) else x == '192.0.2.222') | df['dest_ip'].apply(lambda x: '192.0.2.222' in x if isinstance(x, list) else x == '192.0.2.222')
df_222 = df[mask_222]
df_222

In [ ]:
pd.set_option("display.max_colwidth", None)
# Group by event.original, show source/dest IP/port and count
event_group = df.groupby('event.original').agg({
    'src_ip': lambda x: x.iloc[0],
    'dest_ip': lambda x: x.iloc[0],
    'src_port': lambda x: x.iloc[0],
    'dest_port': lambda x: x.iloc[0],
    'event.original': 'count'
}).rename(columns={'event.original': 'count'}).reset_index()
event_group.sort_values('count', ascending=False)